# Exercise 2

## Overview

## Implementation

### Install Dependencies

In [11]:
%pip install wave

Note: you may need to restart the kernel to use updated packages.


### Application

In [1]:
def unary_encode(n: int) -> str:
    return '1' * n + '0'

def binary_encode(n: int, k: int) -> str:
    return format(n, '0{}b'.format(k))

def rice_encode(value:int, k: int) -> str:
    modulus = 2**k
    
    quotient = value // modulus
    remainder = value % modulus

    quotient_code = unary_encode(quotient)
    remainder_code = binary_encode(remainder, k)
    rice_code = quotient_code + remainder_code

    return rice_code

In [2]:
def unary_decode(code: str) -> int:
    return code.count('1')

def binary_decode(code: int) -> int:
    return int(code, 2)

def rice_decode(value:str, k: int) -> int:
    modulus = 2**k

    first_0_index = len(value) - 1
    for index, char in enumerate(value):
        if char == '0':
            first_0_index = index
            break

    quotient_code = value[:first_0_index]
    remainder_code = value[first_0_index:]

    quotient = unary_decode(quotient_code)
    remainder = binary_decode(remainder_code)
    number = quotient * modulus + remainder

    return number

In [135]:
def read_file_as_byte_array(file_path: str):
    with open(file_path, 'rb') as file:
        byte_array = file.read()
    return byte_array

def byte_to_bit_string(byte):
    # Converting byte to bit string and ensuring there are 8 digits
    bit_string = bin(byte)[2:].zfill(8)
    
    return bit_string

def rice_encode_audio_file(input_audio_file_path: str, output_audio_file_path: str, k: int):
    # Load the audio file
    audio_buffer = read_file_as_byte_array(input_audio_file_path)

    # Encoding bytes
    encoded_bit_strings = [ rice_encode(byte, k) for byte in audio_buffer ]
    # Separate bits into groups of 8 for bytes
    encoded_bit_string = "".join(encoded_bit_strings)
    encoded_byte_bit_strings = [ encoded_bit_string[i : i+8] for i in range(0, len(encoded_bit_string), 8) ]
    last_bit_string_padding_size = 8 - len(encoded_byte_bit_strings[-1])
    
    # Convert the binary string to bytes
    i, encoded_audio_buffer = 0, bytearray()
    # Adding the number of 0s added for padding to the beginning of the last encoded bit string
    encoded_audio_buffer.append(last_bit_string_padding_size)
    # Converting bits to bytes
    for bit_string in encoded_byte_bit_strings:
        byte = int(bit_string, 2)
        encoded_audio_buffer.append(byte)

    # Writing encoded audio samples to output file
    with open(output_audio_file_path, 'wb') as output_file:
        output_file.write(encoded_audio_buffer)

def rice_decode_audio_file(input_audio_file_path: str, output_audio_file_path: str, k: int):
    # Load the encoded file
    encoded_buffer = read_file_as_byte_array(input_audio_file_path)
    # Extracting the first byte that contains the number of padding zeroes in the last 
    padding_size = encoded_buffer[0]

    # Converting byte array into bit strings
    encoded_bit_strings = [ byte_to_bit_string(byte, False) for byte in encoded_buffer[1:] ]
    # Removing any zeroes used as padding in the last byte
    encoded_bit_strings[-1] = encoded_bit_strings[-1][padding_size:]
    encoded_bit_string = ''.join(encoded_bit_strings)

    i, decoded_buffer = 0, bytearray()
    encoded_byte_start = 0
    
    while i < len(encoded_bit_string):
        if encoded_bit_string[i] == '0':
            encoded_byte_end = i + k + 1
            decoded_byte = rice_decode(encoded_bit_string[encoded_byte_start : encoded_byte_end], k)
            decoded_buffer.append(decoded_byte)
            
            i = encoded_byte_end
            encoded_byte_start = i
        else:
            i += 1

    # Writing encoded audio samples to output file
    with open(output_audio_file_path, 'wb') as output_file:
        output_file.write(decoded_buffer)

In [133]:
# Encoding both the audio files with two k values
rice_encode_audio_file('./files/Sound1.wav', './output/Sound1_Enc_K_2.ex2', k = 2)
# rice_encode_audio_file('./files/Sound1.wav', './output/Sound1_Enc_K_4.ex2', k = 4)
# rice_encode_audio_file('./files/Sound2.wav', './output/Sound2_Enc_K_2.ex2', k = 2)
# rice_encode_audio_file('./files/Sound2.wav', './output/Sound2_Enc_K_4.ex2', k = 4)

last_bit_string_padding_size=1


In [136]:
rice_decode_audio_file('./Sound1_Enc_K_2.ex2', './Sound1_Enc_Dec_K_2.wav', k = 2)